In [24]:
import os
import json
import requests
import polars as pl
from bs4 import BeautifulSoup

acos_links = BeautifulSoup(
    requests.get("https://acos.cs.vt.edu/").text, "html.parser"
).find_all("a")


def get_id(link):
    return "acos/" + link.split("/")[3] + "/" + link.split("/")[-1]


def get_title(link):
    if "/acos-pcex/acos-pcex-examples/" in link:
        return link.split("/")[-1][:-26]
    else:
        return link.split("/")[-1]


weat_authors = {
    item["id"]: item["author"]["fullname"] for item in json.load(open("./hub.json"))
}

webex_authors = {
    item["rdfID"].replace(".", "_"): item["Author"]
    for item in json.load(open("../prep-acos-webexjson/webex.json"))
}


def get_author(link):
    if "/annotated/acos-annex-" in link:
        return [webex_authors.get(link.split("/")[-1], "unknown") or "unknown"]
    elif "/acos-pcex/acos-pcex-examples/" in link:
        return [weat_authors.get(link[-24:], "unknown") or "unknown"]
    else:
        return ["unknown"]


def get_features(link):
    if "/annotated/acos-annex-" in link:
        return ["AnnEx"]
    elif "/jsparsons/jsparsons-python/" in link:
        return ["JS-Parsons"]
    elif "/jsvee/jsvee-java/" in link:
        return ["JSVEE"]
    elif "/jsvee/jsvee-python/" in link:
        return ["JSVEE"]
    elif "/combo/combo-python/" in link:
        return ["JS-Parsons", "JSVEE"]
    elif "/acos-pcex/acos-pcex-examples/" in link:
        return ["PCEX"]


def get_programming_language(link):
    if "/annotated/acos-annex-" in link:
        return [link.split("/")[-2].replace("acos-annex-", "")]
    elif "/jsparsons/jsparsons-python/" in link:
        return ["python"]
    elif "/jsvee/jsvee-java/" in link:
        return ["java"]
    elif "/jsvee/jsvee-python/" in link:
        return ["python"]
    elif "/combo/combo-python/" in link:
        return ["python"]
    elif "/acos-pcex/acos-pcex-examples/" in link:
        if not os.path.exists("./cache"):
            os.makedirs("./cache")
        if not os.path.exists(f"./cache/{link[-24:]}.json"):
            content = requests.get(
                f"https://acos.cs.vt.edu/static/acos-pcex-examples/data/{link[-24:]}.json?_t=1763767480731"
            ).json()
            json.dump(content, open(f"./cache/{link[-24:]}.json", "w"), indent=2)
        content = json.load(open(f"./cache/{link[-24:]}.json"))
        return [content[0]["language"].lower()]


def get_natural_language(link):
    if "/acos-pcex/acos-pcex-examples/" in link and "_es__" in link:
        return ["es"]
    else:
        return ["en"]


acos = pl.DataFrame(
    [
        {
            "catalog_type": "SLCItem",
            "persistentID": get_id(link.get("href")),
            "platform_name": "acos",
            "title": get_title(link.get("href")),
            "description": "",
            "license": "",
            "author": get_author(link.get("href")),
            "institution": [],
            "keywords": [],
            "features": get_features(link.get("href")),
            "programming_language": get_programming_language(link.get("href")),
            "natural_language": get_natural_language(link.get("href")),
            "iframe_url": f"https://acos.cs.vt.edu{link.get('href')}",
            "protocol": ["SPLICE", "LTI 1.1"],
            "protocol_url": [
                f"https://acos.cs.vt.edu{link.get('href')}",
                f"https://acos.cs.vt.edu{link.get('href').replace('/html/', '/lti/')}",
            ],
        }
        for link in acos_links
        if link.get("href").startswith("/html/")
    ]
).with_columns(
    pl.col("author").cast(pl.List(pl.Utf8)).alias("author"),
    pl.col("institution").cast(pl.List(pl.Utf8)).alias("institution"),
    pl.col("keywords").cast(pl.List(pl.Utf8)).alias("keywords"),
    pl.col("features").cast(pl.List(pl.Utf8)).alias("features"),
    pl.col("programming_language").cast(pl.List(pl.Utf8)).alias("programming_language"),
    pl.col("natural_language").cast(pl.List(pl.Utf8)).alias("natural_language"),
)

acos_weat_ids = (
    acos.filter(pl.col("iframe_url").str.contains("/acos-pcex/acos-pcex-examples/"))
    .with_columns(pl.col("iframe_url").str.split("__").list.get(-1).alias("id"))["id"]
    .unique()
)

acos

catalog_type,persistentID,platform_name,title,description,license,author,institution,keywords,features,programming_language,natural_language,iframe_url,protocol,protocol_url
str,str,str,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str]
"""SLCItem""","""acos/acos-annex-c/helloworld_c""","""acos""","""helloworld_c""","""""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/helloworld_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/helloworld_c""]"
"""SLCItem""","""acos/acos-annex-c/printing_c""","""acos""","""printing_c""","""""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/printing_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/printing_c""]"
"""SLCItem""","""acos/acos-annex-c/printing1_c""","""acos""","""printing1_c""","""""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/printing1_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/printing1_c""]"
"""SLCItem""","""acos/acos-annex-c/printing2_c""","""acos""","""printing2_c""","""""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/printing2_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/printing2_c""]"
"""SLCItem""","""acos/acos-annex-c/calculating1…","""acos""","""calculating1_c""","""""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/calculating1_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/calculating1_c""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SLCItem""","""acos/acos-pcex-examples/while_…","""acos""","""while_loops_j_digits""","""""","""""","[""Mohammad Hassany""]",[],[],"[""PCEX""]","[""java""]","[""en""]","""https://acos.cs.vt.edu/html/ac…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/acos-pcex/acos-pcex-examples/while_loops_j_digits__68bdb309f6cc3d7a9cc097dd"", ""https://acos.cs.vt.edu/lti/acos-pcex/acos-pcex-examples/while_loops_j_digits__68bdb309f6cc3d7a9cc097dd""]"
"""SLCItem""","""acos/acos-pcex-examples/while_…","""acos""","""while_loops_j_digits""","""""","""""","[""Mohammad Hassany""]",[],[],"[""PCEX""]","[""java""]","[""en""]","""https://acos.cs.vt.edu/html/ac…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/acos-pcex/acos-pcex-examples/while_loops_j_digits__664e4b9391363872f0ba37d2"", ""https://acos.cs.vt.edu/lti/acos-pcex/acos-pcex-examples/while_loops_j_digits__664e4b9391363872f0ba37d2""]"
"""SLCItem""","""acos/acos-pcex-examples/while_…","""acos""","""while_loops_win_percentage""","""""","""""","[""Mohammad Hassany""]",[],[],"[""PCEX""]","[""java""]","[""en""]","""https://acos.cs.vt.edu/html/ac…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/acos-pcex/acos-pcex-examples/while_loops_win_percentage__68bdb308f6cc3d7a9cc09710"", ""https://acos.cs.vt.edu/lti/acos-pcex/acos-pcex-examples/while_loops_win_percentage__68bdb308f6cc3d7a9cc09710""]"


In [25]:
paws = pl.read_json("./catalog.json")
paws = paws[
    list(
        set(paws.columns)
        - set(
            [
                "short_name",
                "problem_statement",
                "code",
                "metadata",
                # "creation_date",
                "author_id",
            ]
        )
    )
]

paws_weat_ids = (
    paws.filter(
        pl.col("preview_url").str.contains(
            "http://pawscomp2.sis.pitt.edu/pcex-authoring/preview/index.html"
        )
    )
    .with_columns(
        pl.col("preview_url")
        .str.replace_all(
            "http://pawscomp2.sis.pitt.edu/pcex-authoring/preview/index.html", ""
        )
        .str.replace_all(
            "\\?load=http://pawscomp2.sis.pitt.edu/pcex-authoring/api/hub/", ""
        )
        .str.split("?")
        .list.get(0)
        .alias("preview_url")
    )["preview_url"]
    .unique()
)

paws = (
    paws.filter(
        ~paws["provider_id"].is_in(
            [
                "webex",
                "pcex",
                "pcex_ch",
                "pcex_activity",
                "opendsa_slideshows",
                "opendsa_problems",
                "readingmirror",
            ]
        )
    )
    .with_columns(
        pl.concat_list([pl.col("provider_name"), pl.col("type")]).alias("features"),
        pl.concat_list([pl.col("url")]).alias("protocol_url"),
        pl.concat_list([pl.col("domain_name").str.to_lowercase()]).alias(
            "programming_language"
        ),
        pl.concat_list([pl.col("author_name")]).alias("author"),
    )
    .rename(
        {
            "name": "title",
            "preview_url": "iframe_url",
            "id": "persistentID",
        }
    )
    .with_columns(
        pl.struct("persistentID", "provider_id")
        .map_elements(
            lambda row: row["provider_id"] + "." + str(row["persistentID"]),
            return_dtype=pl.Utf8,
        )
        .alias("persistentID"),
        pl.lit("SLCItem").alias("catalog_type"),
        pl.col("provider_id").alias("platform_name"),
        pl.lit("").alias("license"),
        pl.lit([]).alias("institution"),
        pl.lit([]).alias("keywords"),
        pl.struct("author", "creation_date")
        .map_elements(
            lambda row: (
                ["es"]
                if (
                    "jordan" in row["author"][0].lower()
                    and row["creation_date"] >= "2025-06-09T10:31:07.000Z"
                )
                or (
                    "kamil" in row["author"][0].lower()
                    and row["creation_date"] >= "2025-05-12T15:19:48.000Z"
                )
                or (
                    "isaac" in row["author"][0].lower()
                    and row["creation_date"] >= "2025-06-10T08:49:49.000Z"
                )
                else ["en"]
            ),
            return_dtype=pl.List(pl.Utf8),
        )
        .alias("natural_language"),
        pl.lit(["PITT"]).alias("protocol"),
    )[acos.columns]
)

paws

catalog_type,persistentID,platform_name,title,description,license,author,institution,keywords,features,programming_language,natural_language,iframe_url,protocol,protocol_url
str,str,str,str,str,str,list[str],list[null],list[null],list[str],list[str],list[str],str,list[str],list[str]
"""SLCItem""","""pcrs.4021""","""pcrs""","""Calcular promedio de dos numer…","""Program Construction Problems""","""""","[""Kamil Akhuseyinoglu""]",[],[],"[""PCRS"", ""Programming Course Resource System (PCRS)""]","[""python""]","[""es""]","""https://pcrs.utm.utoronto.ca/m…","[""PITT""]","[""https://pcrs.utm.utoronto.ca/mgrids/problems/python/337/embed?act=PCRS&sub=py_avg_two_int_es""]"
"""SLCItem""","""pcrs.4020""","""pcrs""","""Calcular factura mensual (ES)""","""Program Construction Problems""","""""","[""Kamil Akhuseyinoglu""]",[],[],"[""PCRS"", ""Programming Course Resource System (PCRS)""]","[""python""]","[""es""]","""https://pcrs.utm.utoronto.ca/m…","[""PITT""]","[""https://pcrs.utm.utoronto.ca/mgrids/problems/python/336/embed?act=PCRS&sub=py_month_bill_es""]"
"""SLCItem""","""pcrs.4019""","""pcrs""","""Calcular los segundos en n dia…","""Program Construction Problems""","""""","[""Kamil Akhuseyinoglu""]",[],[],"[""PCRS"", ""Programming Course Resource System (PCRS)""]","[""python""]","[""es""]","""https://pcrs.utm.utoronto.ca/m…","[""PITT""]","[""https://pcrs.utm.utoronto.ca/mgrids/problems/python/335/embed?act=PCRS&sub=py_second_in_days_es""]"
"""SLCItem""","""pcrs.4018""","""pcrs""","""Condicion: es impar (ES)""","""Program Construction Problems""","""""","[""Kamil Akhuseyinoglu""]",[],[],"[""PCRS"", ""Programming Course Resource System (PCRS)""]","[""python""]","[""es""]","""https://pcrs.utm.utoronto.ca/m…","[""PITT""]","[""https://pcrs.utm.utoronto.ca/mgrids/problems/python/216/embed?act=PCRS&sub=py_odd_condition_es""]"
"""SLCItem""","""pcrs.4017""","""pcrs""","""Contar Mayusculas (ES)""","""Program Construction Problems""","""""","[""Kamil Akhuseyinoglu""]",[],[],"[""PCRS"", ""Programming Course Resource System (PCRS)""]","[""python""]","[""es""]","""https://pcrs.utm.utoronto.ca/m…","[""PITT""]","[""https://pcrs.utm.utoronto.ca/mgrids/problems/python/334/embed?act=PCRS&sub=py_upper_case_es""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SLCItem""","""sqlknot.393""","""sqlknot""","""Sub-Queries question1""","""Sub-Queries question1""","""""","[""Administrator""]",[],[],"[""SQL-KnoT"", ""Question""]","[""sql""]","[""en""]",null,"[""PITT""]","[""http://adapt2.sis.pitt.edu/sqlknot/index.html?cid=18&tid=1&svc=progvis""]"
"""SLCItem""","""sqlknot.394""","""sqlknot""","""Sub-Queries question2""","""Sub-Queries question2""","""""","[""Administrator""]",[],[],"[""SQL-KnoT"", ""Question""]","[""sql""]","[""en""]",null,"[""PITT""]","[""http://adapt2.sis.pitt.edu/sqlknot/index.html?cid=18&tid=2&svc=progvis""]"
"""SLCItem""","""sqlknot.395""","""sqlknot""","""Sub-Queries question3""","""Sub-Queries question3""","""""","[""Administrator""]",[],[],"[""SQL-KnoT"", ""Question""]","[""sql""]","[""en""]",null,"[""PITT""]","[""http://adapt2.sis.pitt.edu/sqlknot/index.html?cid=18&tid=3&svc=progvis""]"


In [26]:
items = pl.concat([acos, paws], how="vertical")
items = items.with_columns(
    pl.col("description")
    .map_elements(
        lambda desc: "<<empty>>" if len(desc) < 1 else desc,
        return_dtype=pl.Utf8,
    )
    .alias("description")
)
items

catalog_type,persistentID,platform_name,title,description,license,author,institution,keywords,features,programming_language,natural_language,iframe_url,protocol,protocol_url
str,str,str,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str]
"""SLCItem""","""acos/acos-annex-c/helloworld_c""","""acos""","""helloworld_c""","""<<empty>>""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/helloworld_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/helloworld_c""]"
"""SLCItem""","""acos/acos-annex-c/printing_c""","""acos""","""printing_c""","""<<empty>>""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/printing_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/printing_c""]"
"""SLCItem""","""acos/acos-annex-c/printing1_c""","""acos""","""printing1_c""","""<<empty>>""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/printing1_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/printing1_c""]"
"""SLCItem""","""acos/acos-annex-c/printing2_c""","""acos""","""printing2_c""","""<<empty>>""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/printing2_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/printing2_c""]"
"""SLCItem""","""acos/acos-annex-c/calculating1…","""acos""","""calculating1_c""","""<<empty>>""","""""","[""admin""]",[],[],"[""AnnEx""]","[""c""]","[""en""]","""https://acos.cs.vt.edu/html/an…","[""SPLICE"", ""LTI 1.1""]","[""https://acos.cs.vt.edu/html/annotated/acos-annex-c/calculating1_c"", ""https://acos.cs.vt.edu/lti/annotated/acos-annex-c/calculating1_c""]"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SLCItem""","""sqlknot.393""","""sqlknot""","""Sub-Queries question1""","""Sub-Queries question1""","""""","[""Administrator""]",[],[],"[""SQL-KnoT"", ""Question""]","[""sql""]","[""en""]",null,"[""PITT""]","[""http://adapt2.sis.pitt.edu/sqlknot/index.html?cid=18&tid=1&svc=progvis""]"
"""SLCItem""","""sqlknot.394""","""sqlknot""","""Sub-Queries question2""","""Sub-Queries question2""","""""","[""Administrator""]",[],[],"[""SQL-KnoT"", ""Question""]","[""sql""]","[""en""]",null,"[""PITT""]","[""http://adapt2.sis.pitt.edu/sqlknot/index.html?cid=18&tid=2&svc=progvis""]"
"""SLCItem""","""sqlknot.395""","""sqlknot""","""Sub-Queries question3""","""Sub-Queries question3""","""""","[""Administrator""]",[],[],"[""SQL-KnoT"", ""Question""]","[""sql""]","[""en""]",null,"[""PITT""]","[""http://adapt2.sis.pitt.edu/sqlknot/index.html?cid=18&tid=3&svc=progvis""]"


In [27]:
missing_weat_ids = list(set(paws_weat_ids) - set(acos_weat_ids))
print(
    "paws catalog weat ids that are missing from acos (needs to be added):",
    len(missing_weat_ids),
)

print("total number of weat/pcex items:", len(acos_weat_ids))

weat_hub = json.load(open("./hub.json"))

paws catalog weat ids that are missing from acos (needs to be added): 0
total number of weat/pcex items: 335


In [28]:
weat_bosnian_ids = [
    item["id"]
    for item in weat_hub
    if item["author"]["email"] == "mkamberovi1@etf.unsa.ba"
]

print("number of bosnian weat/pcex contents:", len(weat_bosnian_ids))

items = items.with_columns(
    pl.struct("iframe_url", "natural_language")
    .map_elements(
        lambda row: (
            ["bs"]
            if (
                row["iframe_url"]
                and "/acos-pcex/acos-pcex-examples/" in row["iframe_url"]
                and row["iframe_url"].split("__")[-1] in weat_bosnian_ids
            )
            else row["natural_language"]
        ),
        return_dtype=pl.List(pl.Utf8),
    )
    .alias("natural_language")
)

number of bosnian weat/pcex contents: 11


In [29]:
items = items.with_columns(
    pl.struct("iframe_url", "protocol_url")
    .map_elements(
        lambda row: (
            (row["protocol_url"][0] + "&sid=demo&usr=demo&grp=demo")
            if "http://adapt2.sis.pitt.edu/sqlknot/" in row["protocol_url"][0]
            else row["iframe_url"]
        ),
        return_dtype=pl.Utf8,
    )
    .alias("iframe_url")
)

In [30]:
display(
    "items with not_null iframe-url",
    items.filter(pl.col("iframe_url").is_not_null())
    .group_by("platform_name")
    .agg(pl.len().alias("count")),
)

display(
    "items with null iframe-url",
    items.filter(pl.col("iframe_url").is_null())
    .group_by("platform_name")
    .agg(pl.len().alias("count")),
)

'items with not_null iframe-url'

platform_name,count
str,u32
"""sqlknot""",108
"""pcrs""",473
"""quizpet""",194
"""acos""",1613
"""quizjet""",231
"""parsons""",207
"""animatedexamples""",165


'items with null iframe-url'

platform_name,count
str,u32
"""codecheck""",3
"""codelab""",1
"""codeworkout""",23
"""quizjet""",1
"""ctat""",1
"""codeocean""",1
"""pcrs""",2
"""quizpet""",21


In [31]:
items = items.filter(pl.col("iframe_url").is_not_null())

In [32]:
def fix_duplicate_features(val):
    val = val.to_list()
    if val == ["AnimatedExamples", "Animated Example"]:
        return ["Animated Example"]
    elif val == ["Parsons Problems", "Parsons Problem"]:
        return ["Parsons Problem"]
    elif val == ["PCRS", "Programming Course Resource System (PCRS)"]:
        return ["Programming Course Resource System (PCRS)"]
    else:
        return val


items = items.with_columns(
    pl.col("features")
    .map_elements(fix_duplicate_features, return_dtype=pl.List(pl.Utf8))
    .alias("features")
)

In [33]:
authors = json.load(open("./authors.json"))
items = items.with_columns(
    pl.col("author")
    .map_elements(
        lambda val: [authors.get(v.strip(), v.strip()) for v in val],
        return_dtype=pl.List(pl.Utf8),
    )
    .alias("author")
)

In [34]:
for platform_name in items["platform_name"].unique():
    items.filter(pl.col("platform_name") == platform_name).to_pandas().to_json(
        f"./splice_{platform_name}.json", orient="records", indent=2
    )